In [ ]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()
sys.path.append(os.getenv("PROJECT_PATH"))
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from logs.logger import Logger
from urllib3.util.retry import Retry
from core import psycopg2_database as db
from requests.adapters import HTTPAdapter
from core.script_monitor import ScriptMonitor

monitor = ScriptMonitor('Screeners', database_name='officefield_local')

success_logger = Logger(log_type='success')
info_logger = Logger(log_type='info')
error_logger = Logger(log_type='error')

table_name = 'stock_screeners'

method_url_dict = {
    # 'original': "https://finviz.com/screener.ashx?v=111&t=MSFT,AAPL,TSLA,ORCL,AMZN,GOOGL,LMT,DELL,HPQ,META,NVDA,BABA,AVGO,ASML,ADBE,CSCO,CRM,AMD,QCOM,INTC,IBM,SONY",
    'gainer': 'https://finviz.com/screener.ashx?v=150&s=ta_topgainers&o=-change',
    'loser': 'https://finviz.com/screener.ashx?v=111&s=ta_toplosers&o=change',
    'unusual_volume': 'https://finviz.com/screener.ashx?v=111&s=ta_unusualvolume&o=-change',
    'insider_invested': 'https://finviz.com/screener.ashx?v=111&f=fa_salesqoq_pos,sh_insttrans_o30&ft=4&o=-volume',
    'insider_bought': 'https://finviz.com/screener.ashx?v=110&s=it_latestbuys',
    'active': 'https://finviz.com/screener.ashx?v=111&s=ta_mostactive&o=-change',
    'overbought': 'https://finviz.com/screener.ashx?v=111&s=ta_overbought&f=sh_avgvol_o100,sh_price_u20,ta_averagetruerange_o0.25&ft=4',
    'swings': 'https://finviz.com/screener.ashx?v=111&f=cap_midunder,fa_epsqoq_o5,fa_salesqoq_o5,sh_instown_u10,sh_price_u5,ta_averagetruerange_o0.25,ta_sma20_pa,ta_sma200_pa,ta_sma50_pa&ft=4',
    'bouncer': 'https://finviz.com/screener.ashx?v=111&f=sh_avgvol_o200,sh_price_u10,ta_averagetruerange_o0.25,ta_change_d,ta_pattern_horizontal,ta_perf_ddown,ta_perf2_d5u&ft=4',
    'bullish': 'https://finviz.com/screener.ashx?v=111&f=sh_curvol_o200,sh_price_u10,ta_change_u,ta_changeopen_u,ta_perf_dup,ta_perf2_d5o&ft=4&o=-volume',
    'bearish': 'https://finviz.com/screener.ashx?v=111&f=fa_epsqoq_neg,fa_roe_neg,fa_salesqoq_neg,sh_insiderown_low,sh_price_u10,ta_sma200_pb,ta_sma50_pb&ft=4',
    'shorted': 'https://finviz.com/screener.ashx?v=111&f=sh_avgvol_o500,sh_relvol_o1.5,sh_short_o30&o=-volume',
    'new_high': 'https://finviz.com/screener.ashx?v=110&s=ta_newhigh',
    'upgrade': 'https://finviz.com/screener.ashx?v=110&s=n_upgrades',
    'downgrade': 'https://finviz.com/screener.ashx?v=110&s=n_downgrades',
    # 'earningBefore': 'https://finviz.com/screener.ashx?v=110&s=n_earningsbefore',
    'momentum': 'https://finviz.com/screener.ashx?v=111&f=fa_epsqoq_o10,fa_salesqoq_o10,sh_avgvol_o500,sh_relvol_o1.5,ta_highlow52w_b0to10h,ta_perf_1wup,ta_rsi_ob60,ta_sma200_pa&ft=4&o=ticker'
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
    "accept": "text/html",
    "cache-control": "no-cache",
    "pragma": "no-cache",
    "sec-ch-ua": '"Chromium";v="106", "Google Chrome";v="106", "Not;A=Brand";v="99"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1"
}

session = requests.Session()
retry = Retry(connect=3, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

def getFullResponse(url, count=0):
    response = requests.get(url, headers=headers)
    # df = pd.read_html(response.text, attrs={'class': 'styled-table-new is-rounded is-tabular-nums w-full screener_table'})
    try:
        df = pd.read_html(response.text, attrs={'class': 'styled-table-new is-rounded is-tabular-nums w-full screener_table'})
    except ValueError as e:
        print(f"Error while parsing tables: {e}")
        return None
    tickers = df[0]['Ticker']
    ticker1 = []
    for ticker in tickers:
        ticker1.append(ticker)
    return ticker1
    
def ingestScreener(database_name, symbols_in_db):
    try:
        result = {}
        count = 0
        total_count = 0
        for method, url in method_url_dict.items():
            print(method)
            range = 1
            result[method] = []
            while True:
                symbols = getFullResponse(f"{url}&r={range}")
                if symbols and len(symbols) > 1:
                    for symbol in symbols:
                        total_count += 1
                        if symbol in symbols_in_db:
                            count += 1
                            if symbol in result[method]:
                                break
                            else:
                                result[method].append(symbol)  
                    info_logger.log(f"symbols exist in db {count} out of {total_count} in in {time.time() - start:.2f} seconds")       
                else:
                    break
                range += 20

        symbols = set()
        for key, values in result.items():
            symbols.update(values)
        data = []
        columns = list(result.keys())
        columns.insert(0, 'symbol')
        for symbol in symbols:
            row = [symbol]
            for key in columns[1:]:
                if symbol in result.get(key, []):
                    row.append(True)
                else:
                    row.append(False)
            data.append(row)
        if data:
            db.truncateTable(database_name, table_name)
            success_logger.log(f"Successfully truncate data in database in {time.time() - start:.2f} seconds")
            db.insert(database_name=database_name, table_name=table_name, columns=columns, values=data)
            success_logger.log(f"Successfully saved data to database in {time.time() - start:.2f} seconds")

    except Exception as e:
        error_logger.log(e)

if __name__ == "__main__":
    try:
        """
            Script Args:
                database_name: see database file for names
        """
        
        start = time.time()
        id = monitor.running()
        list_operations = sys.argv
        database_name = list_operations[1]
        query = """
                    SELECT symbol FROM stock_symbols
                """
        symbols = db.getDataFromQuery(database_name=database_name, query=query)
        success_logger.log(f"Successfully get data from database {time.time() - start:.2f} seconds")
        symbols = [symbol[0] for symbol in symbols]
        ingestScreener(database_name, symbols)
        success_logger.log(f'Successfully ingested in {time.time() - start:.2f} seconds')
        monitor.ended(id, 'Completed')
    except Exception as e:
        monitor.ended(id, 'Error')
        error_logger.log(e)

## Scrap Finviz table using read_html

In [ ]:
#libraries to import 
import os
import sys
import time
import requests
import pandas as pd
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [ ]:
#add header and links
method_url_dict = {"Write all links with their corresponding keys where the table exists."}
headers = { "Write Header"}

#for multi retries
session = requests.Session()
retry = Retry(connect=3, backoff_factor=1)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [ ]:
# Main function for scraping tables, link by link.
def getFullResponse(url, count=0):
    response = requests.get(url, headers=headers)
    # df = pd.read_html(response.text, attrs={'class': 'styled-table-new is-rounded is-tabular-nums w-full screener_table'})
    try:
        df = pd.read_html(response.text, attrs={'class': 'styled-table-new is-rounded is-tabular-nums w-full screener_table'})
    except ValueError as e:
        print(f"Error while parsing tables: {e}")
        return None
    tickers = df[0]['Ticker']
    all_tickers = []
    for ticker in tickers:
        ticker1.append(ticker)
    return all_tickers

In [ ]:
#Get all the data and transform it as needed.
def ingestScreener(database_name, symbols_in_db):
    try:
        result = {}
        count = 0
        total_count = 0
        for method, url in method_url_dict.items():
            range = 1
            result[method] = []
            while True:
                symbols = getFullResponse(f"{url}&r={range}")
                if symbols and len(symbols) > 1:
                    for symbol in symbols:
                        total_count += 1
                        #if symbol available update it.
                        if symbol in symbols_in_db:
                            count += 1
                            if symbol in result[method]:
                                break
                            else:
                            #else append symbol too
                                result[method].append(symbol)  
                    info_logger.log(f"symbols exist in db {count} out of {total_count} in in {time.time() - start:.2f} seconds")       
                else:
                    break
                range += 20

        symbols = set()
        for key, values in result.items():
            symbols.update(values)
        data = []
        columns = list(result.keys())
        columns.insert(0, 'symbol')
        for symbol in symbols:
            row = [symbol]
            for key in columns[1:]:
                if symbol in result.get(key, []):
                    row.append(True)
                else:
                    row.append(False)
            data.append(row)
        if data:
            db.truncateTable(database_name, table_name)
            success_logger.log(f"Successfully truncate data in database in {time.time() - start:.2f} seconds")
            db.insert(database_name=database_name, table_name=table_name, columns=columns, values=data)
            success_logger.log(f"Successfully saved data to database in {time.time() - start:.2f} seconds")

    except Exception as e:
        error_logger.log(e)

In [ ]:
if __name__ == "__main__":
    try:
        """
            Script Args:
                database_name: see database file for names
        """
        
        start = time.time()
        #To add an argument when running a script, we use sys.argv to pass the argument.
        list_operations = sys.argv
        database_name = list_operations[1]
        #to get symbols from database
        query = """
                    SELECT symbol FROM stock_symbols
                """
        symbols = db.getDataFromQuery(database_name=database_name, query=query)
        success_logger.log(f"Successfully get data from database {time.time() - start:.2f} seconds")
        symbols = [symbol[0] for symbol in symbols]
        ingestScreener(database_name, symbols)
        success_logger.log(f'Successfully ingested in {time.time() - start:.2f} seconds')
    except Exception as e:
        error_logger.log(e)